In [80]:
# imports
# system related
import sys
import subprocess

# basics
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep

# snap7 and logging
import snap7 as s7


print('imports succesful')

imports succesful


## Connecting to plc

In [81]:
# connecting to plc

plc = s7.client.Client()
plc.connect('192.168.0.19', 0, 1, )  # ip, rack, slot

print(f'connection status: {plc.get_connected()}')

connection status: True


## Defining get & set functions for plc variables

In [82]:
# Defining read/write functions

def get_bool(db_number, start_offset, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    value = s7.util.get_bool(data, 0, bit_offset)
    # print(f'db number: {db_number}, bit: {start_offset}, has boolean value: {value}')
    return value
    
def set_bool(db_number, start_offset, value, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    s7.util.set_bool(data, 0, bit_offset, value)
    plc.db_write(db_number, start_offset, data)
    return None


def get_real(db_number, start_offset):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    real = s7.util.get_real(data, 0)
    return real

def set_real(db_number, start_offset, value):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    s7.util.set_real(data, 0, value)
    plc.db_write(db_number, start_offset, data)
    return None



    


## Creating RPMs to loop thru

In [83]:
# creating random values to loop thru

############ CHECK THESE VAls!!

min_rpm = 22
max_rpm = 37
d_rpm =   1
num_values = 1111

rpm_interval = 5  # s


rpms = [min_rpm]


while len(rpms) < num_values:
    # create random rpm value
    rpm = int(np.round(np.random.uniform(min_rpm, max_rpm),0))
    # append if difference is high enough. 
    if abs(rpm-rpms[-1]) >= d_rpm:
        rpms.append(rpm)

print_cols = 50
print(f'RPMs to loop thru, in pairs of {print_cols}:\n') 

print(f'  {1} to  {print_cols+1}| ', end='')
for i, val in enumerate(rpms):
    if (i+1)%print_cols != 0:
        print(val, end=' ')
    else: 
        if i == 49:
            print(val, end =f'\n {i+1} to {i+print_cols+1}| ')
        else:
            if i+1 == num_values:
                print(val)
            else:
                print(val, end =f'\n{i+1} to {i+print_cols+1}| ')
print(f'\nExpected run time: {len(rpms)*rpm_interval/60} minutes')


RPMs to loop thru, in pairs of 50:

  1 to  51| 22 34 37 32 33 34 35 25 35 23 28 30 31 29 34 32 29 25 27 30 25 32 35 25 34 23 22 30 34 32 30 24 32 29 28 27 23 30 36 25 28 33 27 25 24 37 24 30 29 33
 50 to 100| 36 37 36 24 31 27 26 33 35 22 33 36 35 22 26 29 24 35 29 25 32 36 28 30 32 26 27 36 28 23 30 36 24 23 28 35 32 26 22 32 30 29 25 22 24 28 25 33 35 36
100 to 150| 35 34 31 35 23 31 22 28 29 27 31 22 23 31 36 32 25 22 34 28 33 31 35 36 28 31 36 22 27 37 25 32 37 22 36 34 31 32 24 36 26 32 28 32 35 27 36 23 32 25
150 to 200| 24 37 22 31 32 29 28 27 34 24 27 36 32 34 22 25 23 22 30 33 31 27 33 28 29 35 23 37 33 35 33 35 25 31 30 24 25 23 36 30 25 24 23 32 34 33 35 28 27 23
200 to 250| 34 22 35 34 36 23 34 32 23 32 29 36 31 32 28 31 29 27 35 27 26 36 31 27 24 31 28 23 25 23 31 29 34 33 24 29 31 33 31 37 32 25 30 23 31 22 31 30 29 24
250 to 300| 29 35 25 30 26 33 35 29 24 31 29 31 35 33 32 25 33 34 23 32 29 25 28 32 34 24 28 25 28 24 36 26 22 24 27 35 30 24 36 24 23 34 28 33 25 33 24 3

## Defining DB number and start offsets for the variables

In [84]:
db_py = 50

# defining start_offsets (so) for the variables in the DB50
so_rpm = 0
so_hz1 = 4
so_hz2 = 8
so_hz3 = 12
so_hz4 = 16


In [86]:
# for loop true random rpms

try:
    for i, rpm in enumerate(rpms):
        set_real(db_py, so_rpm, rpm)
        print(f'Progress: |{int(100*(i+1)/len(rpms))*"█" + int((100*(1-(i+1)/len(rpms))))*"-"}| {np.round((i+1)/len(rpms)*100 ,1)}% |{i+1}/{len(rpms)} | ETA: {np.round((len(rpms)-i)*rpm_interval/60,1)} min | RPM: {get_real(db_py, so_rpm)}', end='\r')
        sleep(rpm_interval)
        
except KeyboardInterrupt:
    print('Sending RPMs interupted by keystroke                                                         It came this far: ')



In [ ]:
# start make datalog function for ixon db

In [48]:
# ixon_remake.

db_log = 33

log_so_width = 0
log_so_rpm = 4
log_so_torque = 12
log_so_hz1 = 16
log_so_hz2 = 24
log_so_hz3 = 32
log_so_hz4 = 40

names = []

get_real(db_log, log_so_width)




b' ISO : An error occurred during send TCP : Connection timed out'


RuntimeError: b' ISO : An error occurred during send TCP : Connection timed out'

### another way of unpacking a database data (Thijs)

In [ ]:
import struct
# struct.unpack("b",plc.db_read(db_number, 2, 1)) 

2